Import libraries

In [46]:
import pandas as pd
import numpy as np
import warnings
# отключаем вывод предупреждений в консоль (чтобы она не засорялась)
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.calibration import CalibrationDisplay

Data import

In [38]:
train_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')
original = pd.read_csv('data/credit_risk_dataset.csv')
train_data.head()

,id,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
0,0,37,35000,RENT,0.0,EDUCATION,B,6000,11.49,0.17,N,14,0
1,1,22,56000,OWN,6.0,MEDICAL,C,4000,13.35,0.07,N,2,0
2,2,29,28800,OWN,8.0,PERSONAL,A,6000,8.90,0.21,N,10,0
3,3,30,70000,RENT,14.0,VENTURE,B,12000,11.11,0.17,N,5,0
4,4,22,60000,RENT,2.0,MEDICAL,A,6000,6.92,0.10,N,3,0


EDA

In [39]:
X = train_data.drop('loan_status', axis=1)
y = train_data['loan_status'].copy()

In [40]:
original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32581 entries, 0 to 32580
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   person_age                  32581 non-null  int64  
 1   person_income               32581 non-null  int64  
 2   person_home_ownership       32581 non-null  object 
 3   person_emp_length           31686 non-null  float64
 4   loan_intent                 32581 non-null  object 
 5   loan_grade                  32581 non-null  object 
 6   loan_amnt                   32581 non-null  int64  
 7   loan_int_rate               29465 non-null  float64
 8   loan_status                 32581 non-null  int64  
 9   loan_percent_income         32581 non-null  float64
 10  cb_person_default_on_file   32581 non-null  object 
 11  cb_person_cred_hist_length  32581 non-null  int64  
dtypes: float64(3), int64(5), object(4)
memory usage: 3.0+ MB


In [41]:
# имеются пропущенные значения - заполним их перед применением модели
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
original[['person_emp_length', 'loan_int_rate']] = imputer.fit_transform(original[['person_emp_length', 'loan_int_rate']])

In [42]:
original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32581 entries, 0 to 32580
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   person_age                  32581 non-null  int64  
 1   person_income               32581 non-null  int64  
 2   person_home_ownership       32581 non-null  object 
 3   person_emp_length           32581 non-null  float64
 4   loan_intent                 32581 non-null  object 
 5   loan_grade                  32581 non-null  object 
 6   loan_amnt                   32581 non-null  int64  
 7   loan_int_rate               32581 non-null  float64
 8   loan_status                 32581 non-null  int64  
 9   loan_percent_income         32581 non-null  float64
 10  cb_person_default_on_file   32581 non-null  object 
 11  cb_person_cred_hist_length  32581 non-null  int64  
dtypes: float64(3), int64(5), object(4)
memory usage: 3.0+ MB


In [43]:
oof = {}
test_pred = {}
NUM_FOLD = 5
target = 'loan_status'

In [47]:
def cross_validation(model, label):
    
    train_copy, test_copy, original_copy = train_data.copy(), test_data.copy(), original.copy()
    
    if label in ['cb', 'et', 'rf', 'knn', 'mlp']:
        cat_cols = test_copy.columns.tolist()
        for df in [train_copy, test_copy, original_copy]:
            for col in cat_cols:  
                df[col] = df[col].astype('str').astype('category')
        
    elif label in ['xgb', 'lgbm', 'dart', 'goss']: 
        cat_cols = list(test_copy.select_dtypes(include=['object']).columns)
        for df in [train_copy, test_copy, original_copy]:
            for col in cat_cols:  
                df[col] = df[col].astype('str').astype('category')
                
    X = train_copy.drop([target], axis=1)
    y = train_copy[target]
    X_original = original_copy.drop([target], axis=1)
    y_original = original_copy[target]
        

    val_scores = []
    test_preds_model = []
    oof_model = np.zeros(len(train_data),)
    
    skf = StratifiedKFold(n_splits=NUM_FOLD, shuffle=True, random_state=1)

    for Fold, (train_index, val_index) in enumerate(skf.split(X, y)):
    
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y[train_index], y[val_index]
        
        X_train = pd.concat([X_train, X_original], axis=0)
        y_train = pd.concat([y_train, y_original]) 
    
        model.fit(X_train, y_train)
    
        y_pred = model.predict_proba(X_val)[:, 1]
    
        roc_auc_score_ = roc_auc_score(y_val, y_pred)
    
        print(f'Fold {Fold}: roc_auc_score= {roc_auc_score_:.5f}')
    
        val_scores.append(roc_auc_score_)
        
        oof_model[val_index] = y_pred
        
        test_preds_model.append(model.predict_proba(test_copy)[:, 1])
    # сохраняем предсказания конкретной модели (на валидационном наборе(на каждом фолде) - т.е. она будет полностью заполнена)
    oof[label] = oof_model
    # усредняем предсказания на тестовом наборе по всем фолдам
    test_preds_model = sum(test_preds_model)/len(test_preds_model)
    test_pred[label] = test_preds_model 

    print(f'mean validation roc_auc_score = {np.mean(val_scores):.5f}')
    print(f'std validation roc_auc_score = {np.std(val_scores):.5f}')
    
    plt.figure(figsize=(10, 4))
    plt.suptitle(label, y=1.0, fontsize=20)
    ax = plt.subplot(1, 2, 1)
    plt.title('Calibration')
    CalibrationDisplay.from_predictions(y, oof_model, n_bins=10, strategy='quantile', ax=ax)
    plt.subplot(1, 2, 2)
    plt.title('Histogram')
    plt.hist(oof_model, bins=10)
    plt.show()

In [ ]:


params_lgbm = {
    
    'verbose': -1,
    'random_state': 1,
    'objective': 'binary',
    'n_estimators': 4100,
    'learning_rate': 0.01,
    'colsample_bytree': 0.6,
    'max_depth': 8,
    'max_bin': 5000,
}

model_1 = LGBMClassifier(**params_lgbm)

cross_validation(model_1, 'lgbm')

Fold 0: roc_auc_score= 0.96389
Fold 1: roc_auc_score= 0.96105


KeyboardInterrupt: 

In [52]:
import optuna
from sklearn.model_selection import cross_val_score
from category_encoders import TargetEncoder
from lightgbm import LGBMClassifier
encoder = TargetEncoder(cols=['person_home_ownership', 'loan_intent', 'loan_grade', 'cb_person_default_on_file'])
X_encoded = encoder.fit_transform(train_data.drop('loan_status', axis=1), train_data['loan_status'])
def objective(trial):
    params = {
        'verbose': -1,
        'random_state': 1,
        'data_sample_strategy': 'goss',
        'n_estimators': trial.suggest_int('n_estimators', 1000, 5000),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'max_depth': trial.suggest_int('max_depth', 5, 20),
        'max_bin': trial.suggest_int('max_bin', 200, 5000),
    }
    model = LGBMClassifier(**params)
    score = cross_val_score(model, X_encoded, train_data['loan_status'], cv=5, scoring='roc_auc').mean()
    return score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=5)
best_params = study.best_params

[I 2025-05-25 19:11:26,049] A new study created in memory with name: no-name-35e855d7-00aa-4285-bbaa-d27a7e14b4f8
[W 2025-05-25 19:11:41,286] Trial 0 failed with parameters: {'n_estimators': 4241, 'learning_rate': 0.005427355515055516, 'colsample_bytree': 0.7285413976801611, 'max_depth': 17, 'max_bin': 2497} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "d:\ML-task\myvenv\lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "D:\Temp\ipykernel_17560\4282235049.py", line 19, in objective
    score = cross_val_score(model, X_encoded, train_data['loan_status'], cv=5, scoring='roc_auc').mean()
  File "d:\ML-task\myvenv\lib\site-packages\sklearn\utils\_param_validation.py", line 216, in wrapper
    return func(*args, **kwargs)
  File "d:\ML-task\myvenv\lib\site-packages\sklearn\model_selection\_validation.py", line 684, in cross_val_score
    cv_results = cross_validate(
  File "d:\ML-t

KeyboardInterrupt: 